# Polychrom Tutorial: Contact Map Analysis

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/darinddv/polychrom/blob/master/tutorials/03_contact_map_analysis.ipynb)

Welcome to the contact map analysis tutorial! This tutorial focuses on analyzing polymer simulation data and creating Hi-C-like contact maps from polychrom simulations.

**What you'll learn:**
- Load and analyze polychrom simulation trajectories
- Generate contact maps from 3D coordinates
- Calculate polymer physics observables
- Compare with experimental Hi-C data
- Advanced contact map analysis techniques

**Prerequisites:**
- Complete the [Basic Polymer Simulation Tutorial](https://colab.research.google.com/github/darinddv/polychrom/blob/master/tutorials/01_basic_polymer_simulation.ipynb)
- Understanding of Hi-C experimental technique (helpful)

**Contact Maps in Chromatin Biology:**
Contact maps visualize the spatial organization of chromatin by showing which genomic regions are in close physical proximity. They're the primary output of Hi-C experiments and crucial for understanding 3D genome organization.

## 1. Installation and Setup

In [ ]:
# Install conda and OpenMM (required for polychrom)
!pip install -q condacolab
import condacolab
condacolab.install()

# Install OpenMM via conda
!conda install -c omnia openmm -y

# Install polychrom dependencies
!pip install numpy scipy h5py pandas joblib matplotlib seaborn cooltools

# Clone and install polychrom
!git clone https://github.com/darinddv/polychrom.git
!cd polychrom && pip install -e .

**⚠️ Important:** After running the installation cell above, **restart the runtime** and continue below.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import sys
from scipy import ndimage
from scipy.spatial.distance import pdist, squareform

# Add polychrom to path
sys.path.append('/content/polychrom')

# Import polychrom modules
import polychrom
from polychrom import forcekits, forces, simulation, starting_conformations
from polychrom.hdf5_format import HDF5Reporter
import polychrom.polymer_analyses
import polychrom.contactmaps

# Set style
plt.style.use('default')
sns.set_palette("husl")

print("All imports successful!")
print(f"NumPy version: {np.__version__}")
print(f"Matplotlib version: {plt.matplotlib.__version__}")

## 2. Generate Sample Data

First, let's run a quick simulation to generate trajectory data for analysis.

In [ ]:
# Simulation parameters
N = 1000  # Number of monomers
num_blocks = 20
steps_per_block = 1000

print(f"Running simulation: {N} monomers, {num_blocks} blocks")

# Create initial conformation
polymer = starting_conformations.grow_cubic(N, boxSize=15)

# Setup simulation
output_dir = "contact_analysis_data"
os.makedirs(output_dir, exist_ok=True)

reporter = HDF5Reporter(folder=output_dir, max_data_length=50, overwrite=True)

sim = simulation.Simulation(
    platform="CPU",
    integrator="variableLangevin",
    temperature=300,
    collision_rate=0.01,
    N=N,
    reporters=[reporter]
)

# Set initial data and add forces
sim.set_data(polymer, center=True)
sim.add_force(forcekits.polymer_chains(sim))
sim.add_force(forces.spherical_confinement(sim, density=0.3, k=5.0))
sim.add_force(forces.polynomial_repulsive(sim, trunc=1.5, radiusMult=1.05))

# Energy minimization
sim.local_energy_minimization()
print("Energy minimization completed")

# Run simulation
trajectory = []
for i in range(num_blocks):
    sim.do_block(steps_per_block)
    current_coords = sim.get_data().copy()
    trajectory.append(current_coords)
    
    if (i + 1) % 5 == 0:
        eK, eP = sim.get_energy()
        print(f"Block {i+1}/{num_blocks}: Ek={eK:.1f}, Ep={eP:.1f}")

reporter.dump_data()
trajectory = np.array(trajectory)

print(f"Simulation completed!")
print(f"Trajectory shape: {trajectory.shape} (blocks, monomers, 3D coordinates)")

## 3. Contact Map Fundamentals

Let's understand what contact maps are and how to calculate them from 3D coordinates.

In [ ]:
def calculate_distance_map(coords):
    """Calculate distance matrix between all pairs of monomers"""
    n = len(coords)
    distance_map = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i, n):
            dist = np.linalg.norm(coords[i] - coords[j])
            distance_map[i, j] = dist
            distance_map[j, i] = dist
    
    return distance_map

def distance_to_contact_map(distance_map, cutoff=2.0):
    """Convert distance map to binary contact map"""
    return (distance_map < cutoff).astype(int)

def contact_probability_map(distance_map, cutoff=2.0):
    """Convert distance to contact probability using exponential decay"""
    return np.exp(-distance_map / cutoff)

# Use the last frame for analysis
final_coords = trajectory[-1]

# Calculate different representations
distance_map = calculate_distance_map(final_coords)
contact_map_binary = distance_to_contact_map(distance_map, cutoff=2.0)
contact_map_prob = contact_probability_map(distance_map, cutoff=2.0)

# Visualize different representations
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Distance map
im1 = axes[0].imshow(distance_map, cmap='viridis_r', origin='lower')
axes[0].set_title('Distance Map')
axes[0].set_xlabel('Monomer index')
axes[0].set_ylabel('Monomer index')
plt.colorbar(im1, ax=axes[0], label='Distance (units)')

# Binary contact map
im2 = axes[1].imshow(contact_map_binary, cmap='Reds', origin='lower')
axes[1].set_title('Binary Contact Map (cutoff=2.0)')
axes[1].set_xlabel('Monomer index')
axes[1].set_ylabel('Monomer index')
plt.colorbar(im2, ax=axes[1], label='Contact (0/1)')

# Probability contact map
im3 = axes[2].imshow(contact_map_prob, cmap='Reds', origin='lower')
axes[2].set_title('Probability Contact Map')
axes[2].set_xlabel('Monomer index')
axes[2].set_ylabel('Monomer index')
plt.colorbar(im3, ax=axes[2], label='Contact probability')

plt.tight_layout()
plt.show()

print(f"Contact map statistics:")
print(f"Total contacts (cutoff=2.0): {np.sum(contact_map_binary) // 2}")
print(f"Contact density: {np.sum(contact_map_binary) / (N * N):.4f}")
print(f"Average distance: {np.mean(distance_map):.2f}")

## 4. Trajectory-Averaged Contact Maps

Real contact maps should be averaged over many conformations to get ensemble averages.

In [ ]:
def calculate_ensemble_contact_map(trajectory, cutoff=2.0, start_frame=10):
    """Calculate contact map averaged over trajectory"""
    n_frames, n_monomers, _ = trajectory.shape
    ensemble_map = np.zeros((n_monomers, n_monomers))
    
    # Skip initial frames for equilibration
    for frame in range(start_frame, n_frames):
        coords = trajectory[frame]
        distance_map = calculate_distance_map(coords)
        contact_map = distance_to_contact_map(distance_map, cutoff)
        ensemble_map += contact_map
    
    # Normalize by number of frames
    ensemble_map = ensemble_map / (n_frames - start_frame)
    return ensemble_map

# Calculate ensemble contact map
ensemble_contacts = calculate_ensemble_contact_map(trajectory, cutoff=2.0, start_frame=10)

# Compare single frame vs ensemble
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Single frame
im1 = axes[0].imshow(contact_map_binary, cmap='Reds', origin='lower', vmin=0, vmax=1)
axes[0].set_title('Single Frame Contact Map')
axes[0].set_xlabel('Monomer index')
axes[0].set_ylabel('Monomer index')
plt.colorbar(im1, ax=axes[0], label='Contact')

# Ensemble averaged
im2 = axes[1].imshow(ensemble_contacts, cmap='Reds', origin='lower', vmin=0, vmax=1)
axes[1].set_title('Ensemble Averaged Contact Map')
axes[1].set_xlabel('Monomer index')
axes[1].set_ylabel('Monomer index')
plt.colorbar(im2, ax=axes[1], label='Contact probability')

# Log scale (Hi-C style)
log_contacts = np.log10(ensemble_contacts + 1e-6)
im3 = axes[2].imshow(log_contacts, cmap='Reds', origin='lower')
axes[2].set_title('Log-scale Contact Map')
axes[2].set_xlabel('Monomer index')
axes[2].set_ylabel('Monomer index')
plt.colorbar(im3, ax=axes[2], label='log10(contact probability)')

plt.tight_layout()
plt.show()

print(f"Ensemble contact statistics:")
print(f"Maximum contact probability: {np.max(ensemble_contacts):.3f}")
print(f"Mean contact probability: {np.mean(ensemble_contacts):.6f}")
print(f"Contact probability at distance 10: {ensemble_contacts[500, 510]:.4f}")
print(f"Contact probability at distance 100: {ensemble_contacts[500, 600]:.4f}")

## 5. Polymer Scaling Analysis

Contact probability in polymers follows characteristic scaling laws. Let's analyze these patterns.

In [ ]:
def calculate_contact_vs_distance(contact_map, max_distance=None):
    """Calculate average contact probability vs genomic distance"""
    n = contact_map.shape[0]
    if max_distance is None:
        max_distance = n // 4
    
    distances = []
    contact_probs = []
    
    for d in range(1, max_distance):
        # Get all pairs at distance d
        contacts = []
        for i in range(n - d):
            contacts.append(contact_map[i, i + d])
        
        if contacts:
            distances.append(d)
            contact_probs.append(np.mean(contacts))
    
    return np.array(distances), np.array(contact_probs)

# Calculate scaling
distances, contact_probs = calculate_contact_vs_distance(ensemble_contacts, max_distance=200)

# Fit power law: P(s) ~ s^(-alpha)
# Use log-log fit
valid_indices = (contact_probs > 0) & (distances > 5)  # Avoid very short distances
valid_distances = distances[valid_indices]
valid_probs = contact_probs[valid_indices]

log_distances = np.log10(valid_distances)
log_probs = np.log10(valid_probs)

# Linear fit in log space
coeffs = np.polyfit(log_distances, log_probs, 1)
alpha = -coeffs[0]  # Scaling exponent
intercept = coeffs[1]

# Generate fit line
fit_distances = np.logspace(np.log10(5), np.log10(200), 50)
fit_probs = 10**(intercept) * fit_distances**(-alpha)

# Plot scaling analysis
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Linear scale
ax1.plot(distances, contact_probs, 'bo-', alpha=0.7, markersize=4)
ax1.set_xlabel('Genomic Distance (monomers)')
ax1.set_ylabel('Contact Probability')
ax1.set_title('Contact Probability vs Distance')
ax1.grid(True, alpha=0.3)

# Log-log scale
ax2.loglog(distances, contact_probs, 'bo', alpha=0.7, markersize=4, label='Data')
ax2.loglog(fit_distances, fit_probs, 'r-', linewidth=2, 
           label=f'Power law fit: α = {alpha:.2f}')
ax2.set_xlabel('Genomic Distance (monomers)')
ax2.set_ylabel('Contact Probability')
ax2.set_title('Scaling Analysis (log-log)')
ax2.grid(True, alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()

print(f"Polymer scaling analysis:")
print(f"Scaling exponent α = {alpha:.3f}")
print(f"")
print(f"Theoretical predictions:")
print(f"  Ideal chain: α = 1.5")
print(f"  Self-avoiding walk: α ≈ 1.8")
print(f"  Compact globule: α ≈ 1.0")
print(f"  Ring polymer: α ≈ 1.0-1.5")
print(f"")
print(f"Your polymer shows {'ideal chain' if 1.4 < alpha < 1.6 else 'self-avoiding' if 1.7 < alpha < 1.9 else 'compact' if alpha < 1.3 else 'unknown'} behavior")

## 6. Advanced Contact Map Analysis

Let's explore more sophisticated analysis techniques used in Hi-C data analysis.

In [ ]:
def calculate_insulation_score(contact_map, window_size=20):
    """Calculate insulation score along the diagonal"""
    n = contact_map.shape[0]
    insulation = np.zeros(n)
    
    for i in range(window_size, n - window_size):
        # Calculate mean contact frequency in upstream-downstream windows
        upstream_downstream = contact_map[i-window_size:i, i:i+window_size]
        insulation[i] = np.mean(upstream_downstream)
    
    return insulation

def calculate_directionality_index(contact_map, window_size=20):
    """Calculate directionality index (bias in upstream vs downstream contacts)"""
    n = contact_map.shape[0]
    directionality = np.zeros(n)
    
    for i in range(window_size, n - window_size):
        # Upstream contacts
        upstream = np.sum(contact_map[i-window_size:i, i:i+window_size])
        # Downstream contacts  
        downstream = np.sum(contact_map[i:i+window_size, i-window_size:i])
        
        total = upstream + downstream
        if total > 0:
            directionality[i] = (upstream - downstream) / total
    
    return directionality

# Calculate advanced metrics
insulation = calculate_insulation_score(ensemble_contacts, window_size=25)
directionality = calculate_directionality_index(ensemble_contacts, window_size=25)

# Plot advanced analysis
fig, axes = plt.subplots(3, 1, figsize=(15, 12))

# Contact map with annotations
im = axes[0].imshow(ensemble_contacts, cmap='Reds', origin='lower', aspect='auto')
axes[0].set_title('Ensemble Contact Map')
axes[0].set_xlabel('Monomer index')
axes[0].set_ylabel('Monomer index')
plt.colorbar(im, ax=axes[0], label='Contact probability')

# Insulation score
positions = np.arange(len(insulation))
axes[1].plot(positions, insulation, 'b-', linewidth=1.5)
axes[1].fill_between(positions, insulation, alpha=0.3)
axes[1].set_title('Insulation Score (identifies domain boundaries)')
axes[1].set_xlabel('Genomic Position')
axes[1].set_ylabel('Insulation Score')
axes[1].grid(True, alpha=0.3)

# Identify potential boundaries (local minima in insulation)
from scipy.signal import find_peaks
# Find valleys (invert and find peaks)
valleys, _ = find_peaks(-insulation[50:-50], height=-np.percentile(insulation[50:-50], 25))
valleys += 50  # Adjust for offset

for valley in valleys:
    axes[1].axvline(valley, color='red', linestyle='--', alpha=0.7)
    axes[0].axvline(valley, color='red', linestyle='--', alpha=0.7)
    axes[0].axhline(valley, color='red', linestyle='--', alpha=0.7)

# Directionality index
axes[2].plot(positions, directionality, 'g-', linewidth=1.5)
axes[2].axhline(0, color='black', linestyle='-', alpha=0.5)
axes[2].fill_between(positions, directionality, alpha=0.3)
axes[2].set_title('Directionality Index (identifies contact asymmetry)')
axes[2].set_xlabel('Genomic Position')
axes[2].set_ylabel('Directionality Index')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Advanced analysis results:")
print(f"Potential domain boundaries detected: {len(valleys)}")
print(f"Boundary positions: {valleys}")
print(f"Average domain size: {np.diff(valleys).mean():.1f} monomers" if len(valleys) > 1 else "")
print(f"Insulation score range: {np.min(insulation[50:-50]):.4f} - {np.max(insulation[50:-50]):.4f}")
print(f"Directionality range: {np.min(directionality[50:-50]):.3f} - {np.max(directionality[50:-50]):.3f}")

## 7. Compartment Analysis

Large-scale chromatin organization can be analyzed using principal component analysis.

In [ ]:
def calculate_compartments(contact_map, resolution=50):
    """Calculate A/B compartments using PCA"""
    # Bin the contact map to lower resolution
    n = contact_map.shape[0]
    n_bins = n // resolution
    
    binned_map = np.zeros((n_bins, n_bins))
    for i in range(n_bins):
        for j in range(n_bins):
            i_start, i_end = i * resolution, (i + 1) * resolution
            j_start, j_end = j * resolution, (j + 1) * resolution
            binned_map[i, j] = np.mean(contact_map[i_start:i_end, j_start:j_end])
    
    # Observed/Expected normalization
    oe_map = np.zeros_like(binned_map)
    for d in range(n_bins):
        if d == 0:
            continue
        diagonal_values = [binned_map[i, i + d] for i in range(n_bins - d) if binned_map[i, i + d] > 0]
        if diagonal_values:
            expected = np.mean(diagonal_values)
            for i in range(n_bins - d):
                if binned_map[i, i + d] > 0:
                    oe_map[i, i + d] = binned_map[i, i + d] / expected
                    oe_map[i + d, i] = oe_map[i, i + d]
    
    # PCA on correlation matrix
    correlation_matrix = np.corrcoef(oe_map)
    # Handle NaN values
    correlation_matrix = np.nan_to_num(correlation_matrix)
    
    # Eigendecomposition
    eigenvalues, eigenvectors = np.linalg.eigh(correlation_matrix)
    
    # First PC (largest eigenvalue)
    pc1 = eigenvectors[:, -1]  # Last column has largest eigenvalue
    
    return pc1, oe_map, binned_map

# Calculate compartments
pc1, oe_map, binned_map = calculate_compartments(ensemble_contacts, resolution=40)

# Plot compartment analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Original contact map
im1 = axes[0,0].imshow(ensemble_contacts, cmap='Reds', origin='lower')
axes[0,0].set_title('Original Contact Map')
axes[0,0].set_xlabel('Monomer index')
axes[0,0].set_ylabel('Monomer index')
plt.colorbar(im1, ax=axes[0,0], shrink=0.7)

# Binned contact map
im2 = axes[0,1].imshow(binned_map, cmap='Reds', origin='lower')
axes[0,1].set_title('Binned Contact Map')
axes[0,1].set_xlabel('Bin index')
axes[0,1].set_ylabel('Bin index')
plt.colorbar(im2, ax=axes[0,1], shrink=0.7)

# O/E contact map
im3 = axes[1,0].imshow(oe_map, cmap='RdBu_r', origin='lower', vmin=0.5, vmax=1.5)
axes[1,0].set_title('Observed/Expected Contact Map')
axes[1,0].set_xlabel('Bin index')
axes[1,0].set_ylabel('Bin index')
plt.colorbar(im3, ax=axes[1,0], shrink=0.7)

# PC1 (compartment signal)
bin_positions = np.arange(len(pc1))
colors = ['red' if x > 0 else 'blue' for x in pc1]
axes[1,1].bar(bin_positions, pc1, color=colors, alpha=0.7)
axes[1,1].axhline(0, color='black', linestyle='-', alpha=0.8)
axes[1,1].set_title('PC1 - A/B Compartments')
axes[1,1].set_xlabel('Bin index')
axes[1,1].set_ylabel('PC1 value')
axes[1,1].grid(True, alpha=0.3)

# Add compartment annotation to O/E map
for i, val in enumerate(pc1):
    color = 'red' if val > 0 else 'blue'
    axes[1,0].axhline(i, color=color, alpha=0.3, linewidth=2)
    axes[1,0].axvline(i, color=color, alpha=0.3, linewidth=2)

plt.tight_layout()
plt.show()

# Compartment statistics
a_compartment = np.sum(pc1 > 0)
b_compartment = np.sum(pc1 < 0)

print(f"Compartment analysis results:")
print(f"A compartment bins: {a_compartment} ({a_compartment/len(pc1)*100:.1f}%)")
print(f"B compartment bins: {b_compartment} ({b_compartment/len(pc1)*100:.1f}%)")
print(f"PC1 range: {np.min(pc1):.3f} to {np.max(pc1):.3f}")
print(f"")
print(f"Interpretation:")
print(f"  Red regions (A compartment): Active, gene-rich regions")
print(f"  Blue regions (B compartment): Inactive, heterochromatic regions")
print(f"  PC1 captures the dominant pattern of genome organization")

## 8. Comparison with Experimental Data

Let's simulate what our contact map might look like as experimental Hi-C data.

In [ ]:
def simulate_hic_noise(contact_map, coverage=1000000, noise_level=0.1):
    """Simulate Hi-C sequencing noise and coverage effects"""
    # Convert probabilities to expected read counts
    expected_counts = contact_map * coverage
    
    # Add Poisson noise (sequencing noise)
    noisy_counts = np.random.poisson(expected_counts)
    
    # Add systematic noise
    systematic_noise = np.random.normal(1.0, noise_level, contact_map.shape)
    noisy_counts = noisy_counts * systematic_noise
    
    # Ensure symmetry
    noisy_counts = (noisy_counts + noisy_counts.T) / 2
    
    # Set diagonal to zero (Hi-C removes adjacent ligation products)
    np.fill_diagonal(noisy_counts, 0)
    
    return noisy_counts.astype(int)

def normalize_hic_data(counts_matrix):
    """Apply ICE normalization (iterative correction)"""
    # Simple bias removal (not full ICE implementation)
    normalized = counts_matrix.astype(float)
    
    # Remove bins with very low coverage
    bin_sums = np.sum(normalized, axis=1)
    valid_bins = bin_sums > np.percentile(bin_sums, 10)
    
    # Iterative correction (simplified)
    for iteration in range(10):
        bin_sums = np.sum(normalized, axis=1)
        bin_sums[bin_sums == 0] = 1  # Avoid division by zero
        correction = np.sqrt(np.mean(bin_sums[valid_bins]) / bin_sums)
        
        # Apply correction
        normalized = normalized * correction[:, np.newaxis]
        normalized = normalized * correction[np.newaxis, :]
    
    return normalized

# Simulate Hi-C data
hic_counts = simulate_hic_noise(ensemble_contacts, coverage=500000, noise_level=0.15)
hic_normalized = normalize_hic_data(hic_counts)

# Plot comparison
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Theoretical contact map
im1 = axes[0,0].imshow(ensemble_contacts, cmap='Reds', origin='lower')
axes[0,0].set_title('Theoretical Contact Map\n(Simulation Truth)')
axes[0,0].set_xlabel('Monomer index')
axes[0,0].set_ylabel('Monomer index')
plt.colorbar(im1, ax=axes[0,0], shrink=0.8)

# Raw Hi-C counts
im2 = axes[0,1].imshow(hic_counts, cmap='Reds', origin='lower')
axes[0,1].set_title('Simulated Hi-C Counts\n(with noise)')
axes[0,1].set_xlabel('Monomer index')
axes[0,1].set_ylabel('Monomer index')
plt.colorbar(im2, ax=axes[0,1], shrink=0.8)

# Normalized Hi-C
im3 = axes[0,2].imshow(hic_normalized, cmap='Reds', origin='lower')
axes[0,2].set_title('Normalized Hi-C Data\n(ICE-like correction)')
axes[0,2].set_xlabel('Monomer index')
axes[0,2].set_ylabel('Monomer index')
plt.colorbar(im3, ax=axes[0,2], shrink=0.8)

# Log-scale comparisons
log_theoretical = np.log10(ensemble_contacts + 1e-6)
log_counts = np.log10(hic_counts + 1)
log_normalized = np.log10(hic_normalized + 1e-6)

axes[1,0].imshow(log_theoretical, cmap='Reds', origin='lower')
axes[1,0].set_title('Log-scale Theoretical')
axes[1,0].set_xlabel('Monomer index')

axes[1,1].imshow(log_counts, cmap='Reds', origin='lower')
axes[1,1].set_title('Log-scale Hi-C Counts')
axes[1,1].set_xlabel('Monomer index')

axes[1,2].imshow(log_normalized, cmap='Reds', origin='lower')
axes[1,2].set_title('Log-scale Normalized')
axes[1,2].set_xlabel('Monomer index')

plt.tight_layout()
plt.show()

# Compare scaling laws
distances_theory, probs_theory = calculate_contact_vs_distance(ensemble_contacts)
distances_hic, probs_hic = calculate_contact_vs_distance(hic_normalized)

plt.figure(figsize=(10, 6))
plt.loglog(distances_theory, probs_theory, 'bo-', alpha=0.7, label='Theoretical', markersize=4)
plt.loglog(distances_hic, probs_hic, 'ro-', alpha=0.7, label='Simulated Hi-C', markersize=4)
plt.xlabel('Genomic Distance')
plt.ylabel('Contact Probability')
plt.title('Scaling Comparison: Theory vs Simulated Hi-C')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Calculate correlation
flat_theory = ensemble_contacts[np.triu_indices_from(ensemble_contacts, k=1)]
flat_hic = hic_normalized[np.triu_indices_from(hic_normalized, k=1)]
correlation = np.corrcoef(flat_theory, flat_hic)[0, 1]

print(f"Hi-C simulation results:")
print(f"Total Hi-C reads: {np.sum(hic_counts):,}")
print(f"Mean reads per bin: {np.mean(hic_counts):.1f}")
print(f"Correlation (theory vs Hi-C): {correlation:.3f}")
print(f"")
print(f"This demonstrates how polymer simulations can:")
print(f"  • Predict contact patterns seen in Hi-C experiments")
print(f"  • Account for experimental noise and biases")
print(f"  • Validate theoretical models against real data")
print(f"  • Guide interpretation of genomic contact maps")

## 9. Summary and Applications

🎉 **Congratulations!** You've mastered contact map analysis with polychrom!

### What You've Learned:

✅ **Contact map fundamentals**: Converting 3D coordinates to 2D contact matrices

✅ **Ensemble averaging**: Proper statistical analysis of polymer trajectories

✅ **Scaling laws**: Understanding polymer physics through contact probability vs distance

✅ **Advanced analysis**: Insulation scores, directionality index, and domain detection

✅ **Compartment analysis**: PCA-based identification of large-scale organization

✅ **Experimental comparison**: Simulating Hi-C noise and normalization procedures

### Key Insights:

🔬 **Polymer physics**: Contact probability follows power laws characteristic of polymer conformations

📊 **Data analysis**: Multiple analysis approaches reveal different organizational levels

🧬 **Biology connection**: Simulations can predict and explain experimental Hi-C patterns

⚖️ **Model validation**: Comparing theory with experiment validates mechanistic models

### Real-World Applications:

🏥 **Disease research**: Compare healthy vs disease chromatin organization

🧬 **Drug discovery**: Predict how compounds affect 3D genome structure

🔬 **Basic research**: Test hypotheses about chromatin organization mechanisms

📈 **Data interpretation**: Understand what Hi-C experiments actually measure

### Next Steps:

🔧 **Tool development**: 
- Build automated contact map analysis pipelines
- Develop statistical tests for comparing contact maps
- Create visualization tools for 3D genome organization

📊 **Advanced analysis**:
- Multi-resolution contact map analysis
- Time-resolved contact map dynamics
- Machine learning approaches to pattern recognition

🧬 **Biological applications**:
- Study specific genomic loci (gene clusters, chromatin domains)
- Model disease-associated structural variants
- Investigate cell-type-specific chromatin organization

### Learn More:

- 📖 [Polychrom Documentation](https://polychrom.readthedocs.io/)
- 🔄 [Loop Extrusion Tutorial](https://colab.research.google.com/github/darinddv/polychrom/blob/master/tutorials/02_loop_extrusion_simulation.ipynb)
- 🧮 [Basic Polymer Tutorial](https://colab.research.google.com/github/darinddv/polychrom/blob/master/tutorials/01_basic_polymer_simulation.ipynb)
- 💻 [GitHub Repository](https://github.com/darinddv/polychrom)
- 📚 [Hi-C Analysis Tools](https://github.com/mirnylab/cooltools)

### Research Opportunities:

The field of 3D genome organization is rapidly evolving. Your skills in contact map analysis position you to contribute to:

- **Computational biology**: Developing new analysis methods
- **Biophysics**: Understanding the physics of chromosome organization
- **Medicine**: Linking 3D genome structure to disease
- **Biotechnology**: Engineering chromatin organization for synthetic biology

---

**Ready for the next challenge?** Explore how these contact map analysis techniques apply to real biological questions in chromatin organization and gene regulation!